In [245]:
%cd D:\GitHub_Repository\Structural-Estimation\bisieco

D:\GitHub_Repository\Structural-Estimation\bisieco


c:\Users\Tsunc\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [246]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS

import random
import warnings

pd.set_option('display.max_columns', None)

## Data Cleaning  

In [247]:
# Arrange Data 
data = pd.read_csv("data/raw/CleanData_20180222.csv", encoding="shift-jis")
data_hh = pd.read_csv("data/raw/HHsize.csv")
data_cpi = pd.read_csv("data/raw/zni2015s.csv", encoding="shift-jis")

df = data[['Maker','Type','Name','Year','Sales','Model','price','kata','weight','FuelEfficiency','HorsePower','overall_length','overall_width','overall_height']]
df = df.rename(columns={'Year': 'year'})

# merge Car Data with HH
df = pd.merge(df, data_hh, on='year')

df["HH"]=df["HH"].str.replace(",","")

data_cpi.drop(index=data_cpi.index[0:5], inplace=True)
data_cpi.rename(columns={'類・品目': 'year', '総合': 'cpi'}, inplace=True)
data_cpi['year'] = data_cpi['year'].astype('int')
data_cpi['cpi'] = data_cpi['cpi'].astype('float')

# merge df with CPI
df = pd.merge(df, data_cpi, on='year')

# drop the null data in FuelEfficiency
df = df.dropna(subset=['FuelEfficiency'])

# realization of price (2016 base)
cpi2016 = data_cpi[data_cpi['year']==2016]
cpi2016 = float(cpi2016.values[0][1])
df['price'] = (df['price'] / (df['cpi'] / cpi2016)) / 100

# Create additional columns
df['size'] = (df['overall_height'] / 1000) * (df['overall_length'] / 1000) * (df['overall_width'] / 1000)
df['hppw'] = df['HorsePower'] / df['weight']

# Create Car ID (based on Model)
df['car_id'] = le.fit_transform(df['Name'])


# Define outside option and market share
df['inside_total'] = df[['year','Sales']].groupby('year').transform('sum')
df['HH'] = df['HH'].astype('int')
df['outside_total'] = df['HH'] - df['inside_total']
df['share'] = df['Sales'] / df['HH']
df['share0'] = df['outside_total'] / df['HH']

## Instrumental Variables  

In [248]:
attribute_cols = ['hppw', 'FuelEfficiency', 'size']

for i in attribute_cols:
    df[f"sum_own_{i}"] = df[['year', 'Maker', i]].groupby(['year','Maker']).transform('sum')[i]
    df[f"sqr_{i}"] = df[i] ** 2
    col_name = "sqr_" + i
    df[f"sqr_sum_own_{i}"] = df[['year', 'Maker', col_name]].groupby(['year','Maker']).transform('sum')[col_name]
for i in attribute_cols:
    df[f"sum_market_{i}"] = df[['year', i]].groupby(['year']).transform('sum')[i]
    col_name = "sqr_" + i
    df[f"sqr_sum_market_{i}"] = df[['year', col_name]].groupby(['year']).transform('sum')[col_name]
    
maker_n = df[['year', 'Maker']].groupby(['year','Maker']).transform('size')
market_n = df[['year']].groupby(['year']).transform('size')

In [249]:
# BLP IV
for i in attribute_cols:
    df[f"iv_BLP_own_{i}"] = df[f"sum_own_{i}"] - df[i]
    df[f"iv_BLP_other_{i}"] = df[f"sum_market_{i}"] - df[i]

# Differentiation IV
for i in attribute_cols:
    df[f"iv_GH_own_{i}"] = (maker_n - 1) * df[f"sqr_{i}"] + (df[f"sqr_sum_own_{i}"] - df[f"sqr_{i}"]) - 2 * df[i] * (df[f"sum_own_{i}"] - df[i])
    df[f"iv_GH_other_{i}"] = (market_n - maker_n) * df[f"sqr_{i}"] + (df[f"sqr_sum_market_{i}"] - df[f"sqr_sum_own_{i}"]) - 2 * df[i] * (df[f"sum_market_{i}"] - df[f"sum_own_{i}"])

## Logit Model

In [250]:
id_list = np.sort(df['car_id'].unique()).tolist()
J = len(id_list)

In [251]:
random_state = 42
random.seed(random_state)
nippyo_id = np.sort(random.sample(id_list, 30))

df_nippyo = df[['year','share','car_id', 'Sales', 'price', 'hppw','FuelEfficiency','size']][df['car_id'].isin(nippyo_id)]
df_nippyo['log_sales'] = np.log(df['Sales'])
df_nippyo['log_price'] = np.log(df['price'])

In [252]:
df['logit_share'] = np.log(df['share']) - np.log(df['share0'])
#fromula logit_share ~ price + hppw + FuelEfficiency + size
X = df[['price','hppw','FuelEfficiency','size']]
y = df['logit_share']
ols = sm.OLS(y, sm.add_constant(X)).fit()

# 2SLS with Differentiation IV
dif = IV2SLS.from_formula("logit_share ~ 1 + hppw + FuelEfficiency + size + \
                [price ~ iv_GH_own_hppw + iv_GH_own_FuelEfficiency + iv_GH_own_size + iv_GH_other_hppw + iv_GH_other_FuelEfficiency + iv_GH_other_size]", df).fit()

In [253]:
# Elasticity Matrix
# car_idが121からずれるのは謎
ex_id = [86,116,150,172]

# Filter dataset for 2016
dt2016 = df[df["year"] == 2016][["price", "share", "car_id"]].sort_values("car_id")
price = dt2016["price"].values
share = dt2016["share"].values
car_id = dt2016['car_id'].values

# Compute elasticities
own_elas = dif.params["price"] * price * (1 - share)
cross_elas = (-1) * dif.params["price"] * price * share

J = len(own_elas)
elas_mat = np.tile(cross_elas, (J, 1))
np.fill_diagonal(elas_mat, own_elas)

mask = np.isin(car_id, ex_id)
result = elas_mat[np.ix_(mask, mask)]
print(result)

[[-1.76455369e+00  1.22041722e-03  1.48175337e-04  1.84509423e-03]
 [ 1.14928839e-03 -8.18688564e-01  1.48175337e-04  1.84509423e-03]
 [ 1.14928839e-03  1.22041722e-03 -9.59449003e-01  1.84509423e-03]
 [ 1.14928839e-03  1.22041722e-03  1.48175337e-04 -6.71750163e-01]]


## Nested Logit

In [254]:
# Create the Nests by car Type (df['Type'])

# summation of attributions within Maker
for i in attribute_cols:
    df[f"sum_own_{i}_nest"] = df[['year', 'Maker','Type', i]].groupby(['year','Maker','Type']).transform('sum')[i]
    df[f"sqr_{i}"] = df[i] ** 2
    col_name = "sqr_" + i
    df[f"sqr_sum_own_{i}_nest"] = df[['year', 'Maker', 'Type', col_name]].groupby(['year','Maker','Type']).transform('sum')[col_name]
# summation of attributions within Market
for i in attribute_cols:
    df[f"sum_market_{i}_nest"] = df[['year','Type', i]].groupby(['year','Type']).transform('sum')[i]
    col_name = "sqr_" + i
    df[f"sqr_sum_market_{i}_nest"] = df[['year', 'Type', col_name]].groupby(['year','Type']).transform('sum')[col_name]

nested_market_n = df.groupby(['year', 'Type'])['Sales'].transform('size')
group_n = df[['year','Maker','Type']].groupby(['year','Maker','Type']).transform('size')

# BLP IV
for i in attribute_cols:
    df[f"iv_BLP_own_{i}_nest"] = df[f"sum_own_{i}_nest"] - df[i]
    df[f"iv_BLP_other_{i}_nest"] = df[f"sum_market_{i}_nest"] - df[i]

df['iv_BLP_own_num_nest'] = group_n - 1
df['iv_BLP_other_num_nest'] = nested_market_n - group_n

# Differentiation IV
for i in attribute_cols:
    df[f"iv_GH_own_{i}_nest"] = (group_n - 1) * df[f"sqr_{i}"] \
                                + (df[f"sqr_sum_own_{i}_nest"] - df[f"sqr_{i}"]) \
                                - 2 * df[i] * (df[f"sum_own_{i}_nest"] - df[i])
    df[f"iv_GH_other_{i}_nest"] = (nested_market_n - group_n) * df[f"sqr_{i}"] \
                                + (df[f"sqr_sum_market_{i}_nest"] - df[f"sqr_sum_own_{i}_nest"]) \
                                - 2 * df[i] * (df[f"sum_market_{i}_nest"] - df[f"sum_own_{i}_nest"])

In [255]:
df['logit_share'] = np.log(df['share']) - np.log(df['share0'])
# Inside share

df['inside_share'] = df['Sales'] / df.groupby(['year', 'Type']).transform('sum')['Sales']
df['log_inside_share'] = np.log(df['inside_share'])

In [ ]:
nested_ols = smf.ols("logit_share ~ 1 + price + log_inside_share + hppw + FuelEfficiency + size", data=df).fit(cov_type='HC3')

nested_blp = IV2SLS.from_formula("logit_share ~ 1 + hppw + FuelEfficiency + size + \
                    [price + log_inside_share ~ iv_BLP_own_hppw_nest + iv_BLP_own_FuelEfficiency_nest + iv_BLP_own_size_nest \
                                                + iv_BLP_other_hppw_nest + iv_BLP_other_FuelEfficiency_nest + iv_BLP_other_size_nest \
                                                + iv_BLP_own_num_nest + iv_BLP_other_num_nest]", data=df).fit()

display(nested_ols.summary())
display(nested_blp.summary)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            logit_share   R-squared:                       0.862
Model:                            OLS   Adj. R-squared:                  0.862
Method:                 Least Squares   F-statistic:                     1673.
Date:                Fri, 04 Apr 2025   Prob (F-statistic):               0.00
Time:                        14:00:19   Log-Likelihood:                -1824.9
No. Observations:                1823   AIC:                             3662.
Df Residuals:                    1817   BIC:                             3695.
Df Model:                           5                                         
Covariance Type:                  HC3                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -7.5571      0.186    -40.633      0.000      -7.922      -7.193
price               -0.3073      0.024    -12.590      0.000      -0.355      -0.259
log_inside_share     0.7820      0.012     66.996      0.000       0.759       0.805
hppw                10.6364      0.864     12.306      0.000       8.942      12.330
FuelEfficiency       0.0551      0.004     13.810      0.000       0.047       0.063
size                 0.1557      0.007     21.132      0.000       0.141       0.170
==============================================================================
Omnibus:                      249.985   Durbin-Watson:                   0.318
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              391.163
Skew:                          -0.944   Prob(JB):                     1.15e-85
Kurtosis:                       4.259   Cond. No.                         898.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:            logit_share   R-squared:                      0.7647
Estimator:                    IV-2SLS   Adj. R-squared:                 0.7641
No. Observations:                1823   F-statistic:                    1399.6
Date:                Fri, Apr 04 2025   P-value (F-stat)                0.0000
Time:                        14:00:19   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                                Parameter Estimates                                 
====================================================================================
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
Intercept           -9.5480     0.2385    -40.039     0.0000     -10.015     -9.0807
hppw                 18.925     1.9617     9.6473     0.0000      15.080      22.770
FuelEfficiency       0.0691     0.0063     11.037     0.0000      0.0568      0.0813
size                 0.2275     0.0122     18.643     0.0000      0.2036      0.2514
price               -0.6542     0.0528    -12.395     0.0000     -0.7576     -0.5507
log_inside_share     0.5951     0.0353     16.881     0.0000      0.5260      0.6642
====================================================================================

Endogenous: price, log_inside_share
Instruments: iv_BLP_own_hppw_nest, iv_BLP_own_FuelEfficiency_nest, iv_BLP_own_size_nest, iv_BLP_other_hppw_nest, iv_BLP_other_FuelEfficiency_nest, iv_BLP_other_size_nest, iv_BLP_own_num_nest, iv_BLP_other_num_nest
Robust Covariance (Heteroskedastic)
Debiased: False
"""

In [ ]:
alpha1 = nested_ols.params['price']
sigma1 = nested_ols.params['log_inside_share']
alpha2 = nested_blp.params['price']
sigma2 = nested_blp.params['log_inside_share']

#self_elasticity
df['own_elas_ols'] = alpha1 * df['price'] * (1 - (sigma1 * df['inside_share']) - (1-sigma1)*df['share'])/(1 - sigma1)
df['own_elas_blp'] = alpha2 * df['price'] * (1 - (sigma2 * df['inside_share']) - (1-sigma2)*df['share'])/(1 - sigma2)

display(df[['own_elas_ols', 'own_elas_blp']].describe())

,own_elas_ols,own_elas_blp
count,1823.000000,1823.000000
mean,-3.521312,-4.049353
std,2.558495,2.937747
min,-17.793584,-20.401493
25%,-4.017510,-4.613132
50%,-2.843831,-3.266397
75%,-1.980574,-2.273343
max,-0.962570,-1.112620


In [ ]:
dt2016 = df[df["year"] == 2016][["price", "share", "car_id", "inside_share", "Type"]].sort_values("car_id")
price = dt2016["price"].values
share = dt2016["share"].values
car_id = dt2016['car_id'].values
inside_share = dt2016['inside_share'].values
group = dt2016['Type'].values

own_elas = alpha2 * price * (1- sigma2 * inside_share - (1-sigma2)*share) / (1-sigma2)

cross_elas_other_group = (-1) * alpha2 * price * share
J = len(own_elas)
elas_mat = np.tile(cross_elas_other_group, (J, 1))
np.fill_diagonal(elas_mat, own_elas)



array([[-4.88275324e+00,  4.31695263e-04,  6.16175532e-04, ...,
         1.62959717e-03,  1.79934407e-03,  2.37857811e-03],
       [ 2.75044651e-04, -5.95803808e+00,  6.16175532e-04, ...,
         1.62959717e-03,  1.79934407e-03,  2.37857811e-03],
       [ 2.75044651e-04,  4.31695263e-04, -6.97206360e+00, ...,
         1.62959717e-03,  1.79934407e-03,  2.37857811e-03],
       ...,
       [ 2.75044651e-04,  4.31695263e-04,  6.16175532e-04, ...,
        -3.04525893e+00,  1.79934407e-03,  2.37857811e-03],
       [ 2.75044651e-04,  4.31695263e-04,  6.16175532e-04, ...,
         1.62959717e-03, -5.10434797e+00,  2.37857811e-03],
       [ 2.75044651e-04,  4.31695263e-04,  6.16175532e-04, ...,
         1.62959717e-03,  1.79934407e-03, -3.55868148e+00]])